In [6]:

from datasets import load_dataset

import argparse

import chromadb
from sentence_transformers import SentenceTransformer

import os
import sys


# project_root = os.path.abspath(os.path.join(os.path.dirname(__file__), '..'))
# 添加项目根目录到sys.path
project_root = '/home/sunmengjie/lpz/vectordb/ragwatermark'
sys.path.append(project_root)
from rag.vectorstore_wh import  VectorStore
from src.utils import load_beir_datasets, load_models,load_json
import argparse
from src.utils import load_json, save_json, find_substrings_containing_sentences,Log,extract_doc_list,extract_doc, file_exist
from src.models import create_model
from src.utils import load_beir_datasets, load_models

import re
import copy 
import torch


# ChromadbPath = '/home/sunmengjie/lpz/vectordb/ragwatermark/chromadb_data'
ChromadbPath = '/data/sunmengjie/chromdb/chromadb_data_copy/chromadb_data'


ModuleNotFoundError: No module named 'rag'

In [1]:
def stat_result(verify_path, stat_path):
    # 判断一个列表里面是不是有1
    def if_have_result(arr):
        for item in arr:
            if item == 1:
                return 1
        return 0


    verify_ids = []
    try:
        verify_ids = load_json(verify_path)
    except FileNotFoundError:
        logger.error(f'{verify_path} does not exist!')

    total = 0
    correct = 0
    wrong = 0
    unknow = 0
    db_search = 0
    wmunit_exist = 0
    wmunit_WE = 0

    for vids in verify_ids:
        total += 1
        db_search += vids[1][1]
        wmunit_exist += vids[1][2]
        wmunit_WE += vids[1][4]
        if vids[2][0] == 0:
            wrong += 1
        elif vids[2][0] == 1:
            correct += 1
        elif vids[2][0] == 2:
            unknow += 1
    for vids in verify_ids:
        total += 1
        db_search = db_search + if_have_result(vids[1][1])
        wmunit_exist =db_search + if_have_result(vids[1][2])
        wmunit_WE += vids[1][4]
        if vids[2][0] == 0:
            wrong += 1
        elif vids[2][0] == 1:
            correct += 1
        elif vids[2][0] == 2:
            unknow += 1


In [ ]:
def wm_verify_run(verify_path, loc_path, doc_path, stat_path,  filter_flag=False, verify_num = 30):
 
    verify_ids = [] ### wmunit, inject ids, search id, contain id,before contain wmunit, after contain wmunit, result, [result detail] 
    try:
        verify_ids = load_json(verify_path)
    except FileNotFoundError:
        logger.error(f'{verify_path} does not exist!')

 

    wmunit_loc_list = []
    try:
        wmunit_loc_list = load_json(loc_path)
    except FileNotFoundError:
        logger.error(f'{loc_path} does not exist!')

    def find_loc(wmunit, ids):
        for wmunit_loc in wmunit_loc_list:
            if wmunit == wmunit_loc[0]:
                for item, idsl in enumerate(wmunit_loc[1]):
                    if ids == idsl :
                        return 1, item
        
        return 0, -1
    
    def find_doc(wmunit, item):
        for wmunit_loc in wmunit_loc_list:
            if wmunit == wmunit_loc[0]:
                return wmunit_loc[2][item] 


    eg0_verify_ids = [['e1','e2','r'],['ids','WT_exsit','WE'],['result','WD']]

     
    rwmunit_loc_list = random.sample(wmunit_loc_list, verify_num)

    for  wmunit_loc in rwmunit_loc_list:
        wmunit = wmunit_loc[0]
        if any(wmunit == vid[0] for vid in verify_ids):
            continue

        eg_verify_ids = copy.deepcopy(eg0_verify_ids)  # 使用深拷贝
        eg_verify_ids[0] = wmunit
        visiter.wm_unit = wmunit
        checker.wm_unit = wmunit

        rag_document, db_ids = visiter.ask_wm()
        db_search =  0
        wmunit_exist = -1
        wmunit_WE = -1
        wmunit_doc_info  = []
        db_search, item = find_loc(wmunit, db_ids)
        wmunit_WE = 0 if find_substrings_containing_sentences(rag_document, wmunit[:2])[1] == 0 else 1

        if item != -1:
            print(item)
            wmunit_doc_info = find_doc(wmunit, item)
            # print(wmunit_doc)
            wmunit_exist = 0 if find_substrings_containing_sentences(wmunit_doc_info[0], wmunit[:2])[1] == 0 else 1
            
                
        
        if filter_flag == True:
            checker.rag_document= find_substrings_containing_sentences(rag_document, wmunit[:2])[0]
        else:
            checker.rag_document= rag_document

        eg_verify_ids[1] = [db_ids, db_search, wmunit_exist, checker.rag_document, wmunit_WE, wmunit_doc_info]
        result = checker.check_wm()
        result_flag = -1
    #  re.compile(r'^(yes|YES|Yes)\.?$')
        if result in ['Yes', 'YES', 'yes', 'Yes.', 'YES.', 'yes.' ]:
            result_flag = 1
        elif result in ['No', 'no', 'NO', 'No.','no.','NO.']:
            result_flag = 0
        else:
            result_flag = 2
        
        eg_verify_ids[2] = [result_flag, result]

        if eg_verify_ids not in verify_ids:  # 确保唯一性
            verify_ids.append(eg_verify_ids)

        save_json(verify_ids, verify_path)

        if len(verify_ids) % 5 == 0:
            stat_result(verify_path, stat_path )
    stat_result(verify_path,stat_path )
    logger.info(f'verify watermark doc len :{len(verify_ids)}')
    return  True


In [ ]:
eval_dataset = 'nfcorpus'
eval_model_code = "contriever"
score_function ='cosine'
collection_name = 'nfcorpus_contriever_cosine'
split='test'

model, c_model, tokenizer, get_emb = load_models( eval_model_code)

if eval_dataset == 'msmarco':
    corpus, queries, qrels = load_beir_datasets('msmarco', 'train')
else:
    corpus, queries, qrels = load_beir_datasets(eval_dataset, split)

vectorstore = VectorStore(model, tokenizer, get_emb, corpus, device, collection_name, use_local=True)  

In [1]:
basepath = '/home/sunmengjie/lpz/vectordb/ragwatermark/output'

wmunit_doc_path = os.path.join(basepath, str(10), 'wmuint_doc.json')
wmunit_inject_path = os.path.join(basepath, str(10), 'wmuint_inject.json')

wmunit_verify_path = os.path.join(basepath, 'vicuna13b', str(10), 'wmuint_verify.json')
wmunit_stat_path = os.path.join(basepath,'vicuna13b', str(10), 'wmuint_stat.json')

wm_verify_run(doc_path=wmunit_doc_path, loc_path=wmunit_inject_path, verify_path=wmunit_verify_path,
                      stat_path=wmunit_stat_path, filter_flag=False)

NameError: name 'os' is not defined